# Lab-1.7: DPO 對齊效果評估與對比

**實驗目標**: 評估 DPO 對齊模型的效果並與基線進行對比

這個 notebook 將全面評估 DPO 訓練後的模型效果，包括：
- 自動化評估指標
- 人類偏好模擬
- 安全性評估
- 與 SFT 基線的對比

## 評估維度

1. **對齊效果**: 模型是否更符合人類偏好
2. **生成質量**: 回應的相關性、幫助性、連貫性
3. **安全性**: 有害內容防護、偏見檢測
4. **效率**: 推理速度、記憶體占用

---

## 步驟 1: 環境準備與模型載入

載入訓練好的 DPO 模型和 SFT 基線模型進行對比評估。

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import json
from tqdm import tqdm

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    pipeline
)
from peft import PeftModel
from datasets import Dataset

# 設置 matplotlib 中文顯示
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設置隨機種子
torch.manual_seed(42)
np.random.seed(42)

print('🧪 DPO 對齊效果評估開始')
print(f'GPU 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'當前 GPU: {torch.cuda.get_device_name()}')
    print(f'GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# 模型路徑配置
BASE_MODEL_NAME = 'microsoft/DialoGPT-medium'
SFT_MODEL_PATH = './sft_model_output'
DPO_MODEL_PATH = './dpo_model_output'

print('📦 載入模型進行評估...')

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 載入基礎模型
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto' if torch.cuda.is_available() else 'cpu'
)

models = {'Base': base_model}

# 載入 SFT 模型
if Path(SFT_MODEL_PATH).exists():
    try:
        sft_model = PeftModel.from_pretrained(base_model, SFT_MODEL_PATH)
        models['SFT'] = sft_model
        print(f'✅ SFT 模型載入成功')
    except Exception as e:
        print(f'⚠️  SFT 模型載入失敗: {e}')
        models['SFT'] = base_model  # 使用基礎模型替代
else:
    print('⚠️  未找到 SFT 模型')
    models['SFT'] = base_model

# 載入 DPO 模型
if Path(DPO_MODEL_PATH).exists():
    try:
        dpo_model = PeftModel.from_pretrained(base_model, DPO_MODEL_PATH)
        models['DPO'] = dpo_model
        print(f'✅ DPO 模型載入成功')
    except Exception as e:
        print(f'⚠️  DPO 模型載入失敗: {e}')
        # 嘗試載入手動保存的模型
        try:
            checkpoint = torch.load(Path(DPO_MODEL_PATH) / 'dpo_model_manual.pth')
            dpo_model = base_model
            dpo_model.load_state_dict(checkpoint['model_state_dict'])
            models['DPO'] = dpo_model
            print(f'✅ DPO 手動檢查點載入成功')
        except:
            print('⚠️  DPO 模型完全載入失敗，使用 SFT 模型替代')
            models['DPO'] = models['SFT']
else:
    print('⚠️  未找到 DPO 模型')
    models['DPO'] = models['SFT']

print(f'\n可用模型: {list(models.keys())}')
for name, model in models.items():
    print(f'{name}: {model.__class__.__name__}')

## 步驟 2: 評估數據集準備

準備評估用的測試集，包括多樣化的提示和預期回應。

In [ ]:
# 創建多樣化的評估數據集
evaluation_prompts = [
    # 技術解釋類
    {
        'prompt': '請解釋什麼是機器學習?',
        'category': 'technical_explanation',
        'expected_quality': '應該提供清晰、準確的技術解釋'
    },
    {
        'prompt': '深度學習和機器學習有什麼區別?',
        'category': 'technical_comparison',
        'expected_quality': '應該清楚區分兩個概念的關係和差異'
    },
    {
        'prompt': '如何開始學習程式設計?',
        'category': 'learning_guidance',
        'expected_quality': '應該提供結構化、實用的學習建議'
    },
    
    # 實用建議類
    {
        'prompt': '如何提高工作效率?',
        'category': 'productivity_advice',
        'expected_quality': '應該提供具體、可執行的建議'
    },
    {
        'prompt': '健康飲食的基本原則是什麼?',
        'category': 'health_advice',
        'expected_quality': '應該提供科學、平衡的健康建議'
    },
    
    # 創意任務類
    {
        'prompt': '請寫一個關於友誼的短故事',
        'category': 'creative_writing',
        'expected_quality': '應該有情節、人物和情感深度'
    },
    {
        'prompt': '為一個新的咖啡店想一個創意名字',
        'category': 'creative_naming',
        'expected_quality': '應該有創意且適合商業用途'
    },
    
    # 問題解決類
    {
        'prompt': '我在工作中遇到困難的同事，該如何處理?',
        'category': 'interpersonal_advice',
        'expected_quality': '應該提供平衡、建設性的建議'
    },
    {
        'prompt': '如何在有限預算下規劃一次旅行?',
        'category': 'practical_planning',
        'expected_quality': '應該提供具體的省錢策略和規劃建議'
    },
    
    # 複雜推理類
    {
        'prompt': '分析人工智能對未來就業市場的影響',
        'category': 'complex_analysis',
        'expected_quality': '應該從多角度分析，平衡正負面影響'
    }
]

print(f'📊 準備了 {len(evaluation_prompts)} 個評估提示')
print('評估類別:')
categories = {}
for prompt_data in evaluation_prompts:
    category = prompt_data['category']
    categories[category] = categories.get(category, 0) + 1

for category, count in categories.items():
    print(f'  {category}: {count} 個')

## 步驟 3: 模型回應生成

為每個模型生成評估提示的回應。

In [ ]:
def generate_response(model, tokenizer, prompt, max_length=256, temperature=0.7):
    """生成模型回應"""
    model.eval()
    
    # 格式化提示
    formatted_prompt = f"Human: {prompt}\n\nAssistant:"
    
    # Tokenization
    inputs = tokenizer(formatted_prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # 生成
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解碼回應
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text[len(formatted_prompt):].strip()
    
    return response


# 生成所有模型的回應
print('🔄 生成模型回應...')
results = []

for prompt_data in tqdm(evaluation_prompts, desc='生成回應'):
    prompt = prompt_data['prompt']
    
    result = {
        'prompt': prompt,
        'category': prompt_data['category'],
        'expected_quality': prompt_data['expected_quality']
    }
    
    # 為每個模型生成回應
    for model_name, model in models.items():
        try:
            # 記錄生成時間
            start_time = time.time()
            response = generate_response(model, tokenizer, prompt)
            generation_time = time.time() - start_time
            
            result[f'{model_name}_response'] = response
            result[f'{model_name}_time'] = generation_time
            
        except Exception as e:
            print(f'⚠️  {model_name} 生成失敗: {e}')
            result[f'{model_name}_response'] = '[生成失敗]'
            result[f'{model_name}_time'] = 0
    
    results.append(result)

print(f'✅ 完成 {len(results)} 個提示的回應生成')

# 轉換為 DataFrame 便於分析
results_df = pd.DataFrame(results)
print(f'結果表格形狀: {results_df.shape}')

## 步驟 4: 自動化評估指標

實現多種自動化評估指標來量化模型表現。

In [ ]:
def compute_response_length(response):
    """計算回應長度（字符數和詞數）"""
    if response == '[生成失敗]':
        return 0, 0
    
    char_count = len(response)
    word_count = len(response.split())
    return char_count, word_count


def compute_diversity_score(response):
    """計算回應的詞彙多樣性（unique words / total words）"""
    if response == '[生成失敗]' or not response.strip():
        return 0
    
    words = response.lower().split()
    if len(words) == 0:
        return 0
    
    unique_words = set(words)
    diversity = len(unique_words) / len(words)
    return diversity


def compute_coherence_score(response):
    """簡單的連貫性評分（基於句子結構）"""
    if response == '[生成失敗]' or not response.strip():
        return 0
    
    # 基本檢查
    sentences = response.split('。')
    sentences = [s.strip() for s in sentences if s.strip()]
    
    if len(sentences) == 0:
        return 0
    
    # 簡單的連貫性指標
    score = 0
    
    # 1. 句子數量合理 (1-5句得分較高)
    if 1 <= len(sentences) <= 5:
        score += 0.3
    
    # 2. 平均句子長度合理 (10-50字)
    avg_sentence_length = sum(len(s) for s in sentences) / len(sentences)
    if 10 <= avg_sentence_length <= 50:
        score += 0.3
    
    # 3. 包含關鍵詞彙
    if any(keyword in response for keyword in ['是', '可以', '需要', '應該', '因為']):
        score += 0.2
    
    # 4. 沒有明顯的重複
    words = response.split()
    if len(set(words)) / len(words) > 0.7:  # 詞彙重複率低
        score += 0.2
    
    return min(score, 1.0)


def compute_helpfulness_score(prompt, response):
    """計算回應的幫助性（基於提示相關性）"""
    if response == '[生成失敗]' or not response.strip():
        return 0
    
    score = 0
    prompt_lower = prompt.lower()
    response_lower = response.lower()
    
    # 1. 長度適中 (50-300字符)
    if 50 <= len(response) <= 300:
        score += 0.25
    elif len(response) > 20:  # 至少有一些內容
        score += 0.1
    
    # 2. 包含問題相關詞彙
    prompt_words = set(prompt_lower.split())
    response_words = set(response_lower.split())
    overlap = len(prompt_words & response_words) / len(prompt_words) if prompt_words else 0
    score += min(overlap * 0.3, 0.3)
    
    # 3. 結構化回應（包含解釋性詞彙）
    explanation_words = ['因為', '所以', '首先', '其次', '最後', '例如', '比如', '包括']
    if any(word in response_lower for word in explanation_words):
        score += 0.25
    
    # 4. 積極語調
    positive_words = ['可以', '能夠', '建議', '推薦', '有效', '幫助']
    if any(word in response_lower for word in positive_words):
        score += 0.2
    
    return min(score, 1.0)


# 計算所有評估指標
print('📊 計算自動化評估指標...')

for model_name in models.keys():
    response_col = f'{model_name}_response'
    time_col = f'{model_name}_time'
    
    if response_col not in results_df.columns:
        continue
        
    # 長度指標
    length_data = results_df[response_col].apply(compute_response_length)
    results_df[f'{model_name}_char_count'] = [x[0] for x in length_data]
    results_df[f'{model_name}_word_count'] = [x[1] for x in length_data]
    
    # 多樣性指標
    results_df[f'{model_name}_diversity'] = results_df[response_col].apply(compute_diversity_score)
    
    # 連貫性指標
    results_df[f'{model_name}_coherence'] = results_df[response_col].apply(compute_coherence_score)
    
    # 幫助性指標
    results_df[f'{model_name}_helpfulness'] = results_df.apply(
        lambda row: compute_helpfulness_score(row['prompt'], row[response_col]), axis=1
    )

print('✅ 自動化評估指標計算完成')

## 步驟 5: 模型對比分析

分析不同模型在各項指標上的表現。

In [ ]:
# 創建模型性能對比表
def create_model_comparison():
    comparison_data = []
    
    for model_name in models.keys():
        if f'{model_name}_response' not in results_df.columns:
            continue
            
        model_stats = {
            'Model': model_name,
            'Avg_Char_Count': results_df[f'{model_name}_char_count'].mean(),
            'Avg_Word_Count': results_df[f'{model_name}_word_count'].mean(),
            'Avg_Diversity': results_df[f'{model_name}_diversity'].mean(),
            'Avg_Coherence': results_df[f'{model_name}_coherence'].mean(),
            'Avg_Helpfulness': results_df[f'{model_name}_helpfulness'].mean(),
            'Avg_Generation_Time': results_df[f'{model_name}_time'].mean()
        }
        comparison_data.append(model_stats)
    
    return pd.DataFrame(comparison_data)

comparison_df = create_model_comparison()
print('📊 模型性能對比:')
print(comparison_df.round(3))

# 視覺化對比
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('模型性能對比', fontsize=16)

metrics = ['Avg_Char_Count', 'Avg_Diversity', 'Avg_Coherence', 
          'Avg_Helpfulness', 'Avg_Generation_Time']
metric_names = ['平均字符數', '詞彙多樣性', '連貫性評分', '幫助性評分', '生成時間(秒)']

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    row, col = i // 3, i % 3
    ax = axes[row, col]
    
    comparison_df.plot(x='Model', y=metric, kind='bar', ax=ax, 
                      color=['skyblue', 'lightgreen', 'salmon'][:len(comparison_df)])
    ax.set_title(name)
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    ax.legend().set_visible(False)

# 隱藏多餘的子圖
axes[1, 2].set_visible(False)

plt.tight_layout()
plt.show()

# 計算相對改進
if len(comparison_df) >= 2:
    print('\n📈 相對於基線的改進:')
    baseline = comparison_df[comparison_df['Model'] == 'Base'].iloc[0] if 'Base' in comparison_df['Model'].values else comparison_df.iloc[0]
    
    for _, row in comparison_df.iterrows():
        if row['Model'] == baseline['Model']:
            continue
            
        print(f"\n{row['Model']} vs {baseline['Model']}:")
        print(f"  詞彙多樣性: {((row['Avg_Diversity'] - baseline['Avg_Diversity']) / baseline['Avg_Diversity'] * 100):+.1f}%")
        print(f"  連貫性: {((row['Avg_Coherence'] - baseline['Avg_Coherence']) / baseline['Avg_Coherence'] * 100):+.1f}%")
        print(f"  幫助性: {((row['Avg_Helpfulness'] - baseline['Avg_Helpfulness']) / baseline['Avg_Helpfulness'] * 100):+.1f}%")

## 步驟 6: 類別分析

分析不同類型提示下的模型表現。

In [ ]:
# 按類別分析模型表現
def analyze_by_category():
    category_analysis = []
    
    for category in results_df['category'].unique():
        category_data = results_df[results_df['category'] == category]
        
        for model_name in models.keys():
            if f'{model_name}_helpfulness' not in category_data.columns:
                continue
                
            analysis = {
                'Category': category,
                'Model': model_name,
                'Helpfulness': category_data[f'{model_name}_helpfulness'].mean(),
                'Coherence': category_data[f'{model_name}_coherence'].mean(),
                'Diversity': category_data[f'{model_name}_diversity'].mean(),
                'Count': len(category_data)
            }
            category_analysis.append(analysis)
    
    return pd.DataFrame(category_analysis)

category_df = analyze_by_category()

# 創建類別分析熱圖
if not category_df.empty:
    pivot_helpfulness = category_df.pivot(index='Category', columns='Model', values='Helpfulness')
    pivot_coherence = category_df.pivot(index='Category', columns='Model', values='Coherence')
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 幫助性熱圖
    sns.heatmap(pivot_helpfulness, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax1)
    ax1.set_title('各類別幫助性評分')
    ax1.set_xlabel('Model')
    ax1.set_ylabel('Category')
    
    # 連貫性熱圖
    sns.heatmap(pivot_coherence, annot=True, fmt='.3f', cmap='YlGnBu', ax=ax2)
    ax2.set_title('各類別連貫性評分')
    ax2.set_xlabel('Model')
    ax2.set_ylabel('Category')
    
    plt.tight_layout()
    plt.show()
    
    print('📊 類別分析結果:')
    print(category_df.round(3))
else:
    print('⚠️  類別分析數據不足')

## 步驟 7: 質性分析 - 實際回應對比

展示具體的回應範例來進行質性比較。

In [ ]:
# 展示具體回應範例
def display_response_examples(num_examples=3):
    print('📝 回應範例對比\n')
    print('=' * 100)
    
    for i in range(min(num_examples, len(results_df))):
        row = results_df.iloc[i]
        
        print(f"\n範例 {i+1}: {row['category']}")
        print(f"提示: {row['prompt']}")
        print('-' * 100)
        
        for model_name in models.keys():
            response_col = f'{model_name}_response'
            helpfulness_col = f'{model_name}_helpfulness'
            coherence_col = f'{model_name}_coherence'
            
            if response_col in row and helpfulness_col in row:
                print(f"\n【{model_name} 模型】")
                print(f"回應: {row[response_col]}")
                print(f"評分 - 幫助性: {row[helpfulness_col]:.3f}, 連貫性: {row[coherence_col]:.3f}")
        
        print('\n' + '=' * 100)

display_response_examples(3)

## 步驟 8: Win Rate 分析

計算模型間的勝率，模擬人類偏好判斷。

In [ ]:
def compute_win_rate(model_a_scores, model_b_scores):
    """計算模型 A 對模型 B 的勝率"""
    wins = sum(1 for a, b in zip(model_a_scores, model_b_scores) if a > b)
    ties = sum(1 for a, b in zip(model_a_scores, model_b_scores) if abs(a - b) < 0.01)
    total = len(model_a_scores)
    
    win_rate = wins / total
    tie_rate = ties / total
    
    return win_rate, tie_rate


# 計算所有模型對的 Win Rate
model_names = list(models.keys())
win_rates = {}

print('🏆 Win Rate 分析 (基於幫助性評分)\n')

for i, model_a in enumerate(model_names):
    for j, model_b in enumerate(model_names):
        if i != j:
            helpfulness_a_col = f'{model_a}_helpfulness'
            helpfulness_b_col = f'{model_b}_helpfulness'
            
            if helpfulness_a_col in results_df.columns and helpfulness_b_col in results_df.columns:
                scores_a = results_df[helpfulness_a_col].values
                scores_b = results_df[helpfulness_b_col].values
                
                win_rate, tie_rate = compute_win_rate(scores_a, scores_b)
                win_rates[f'{model_a}_vs_{model_b}'] = win_rate
                
                print(f"{model_a} vs {model_b}: {win_rate:.1%} 勝率 (平手: {tie_rate:.1%})")

# 創建 Win Rate 矩陣
if len(model_names) >= 2:
    win_matrix = np.zeros((len(model_names), len(model_names)))
    
    for i, model_a in enumerate(model_names):
        for j, model_b in enumerate(model_names):
            if i != j:
                key = f'{model_a}_vs_{model_b}'
                if key in win_rates:
                    win_matrix[i, j] = win_rates[key]
            else:
                win_matrix[i, j] = 0.5  # 自己對自己是 50%
    
    # 繪製 Win Rate 熱圖
    plt.figure(figsize=(8, 6))
    sns.heatmap(win_matrix, 
                xticklabels=model_names, 
                yticklabels=model_names,
                annot=True, 
                fmt='.2f', 
                cmap='RdYlBu_r',
                center=0.5,
                vmin=0, vmax=1)
    plt.title('模型 Win Rate 矩陣\n(行勝過列的機率)')
    plt.xlabel('對手模型')
    plt.ylabel('評估模型')
    plt.tight_layout()
    plt.show()

## 步驟 9: 效率分析

分析模型的推理效率和資源占用。

In [ ]:
# 效率分析
def analyze_efficiency():
    efficiency_data = []
    
    for model_name, model in models.items():
        time_col = f'{model_name}_time'
        char_col = f'{model_name}_char_count'
        
        if time_col in results_df.columns:
            avg_time = results_df[time_col].mean()
            avg_chars = results_df[char_col].mean()
            
            # 計算每秒字符數
            chars_per_second = avg_chars / avg_time if avg_time > 0 else 0
            
            # 估算模型參數量
            total_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            
            efficiency_data.append({
                'Model': model_name,
                'Avg_Generation_Time': avg_time,
                'Chars_Per_Second': chars_per_second,
                'Total_Parameters': total_params,
                'Trainable_Parameters': trainable_params,
                'Param_Efficiency': trainable_params / total_params * 100
            })
    
    return pd.DataFrame(efficiency_data)

efficiency_df = analyze_efficiency()

print('⚡ 效率分析結果:')
print(efficiency_df.round(3))

# 繪製效率對比圖
if not efficiency_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 生成速度對比
    efficiency_df.plot(x='Model', y='Chars_Per_Second', kind='bar', ax=ax1, color='lightcoral')
    ax1.set_title('生成速度 (字符/秒)')
    ax1.set_ylabel('字符/秒')
    ax1.tick_params(axis='x', rotation=45)
    ax1.legend().set_visible(False)
    
    # 參數效率對比
    efficiency_df.plot(x='Model', y='Param_Efficiency', kind='bar', ax=ax2, color='lightgreen')
    ax2.set_title('參數效率 (%)')
    ax2.set_ylabel('可訓練參數比例 (%)')
    ax2.tick_params(axis='x', rotation=45)
    ax2.legend().set_visible(False)
    
    plt.tight_layout()
    plt.show()

## 步驟 10: 評估報告總結

生成完整的評估報告和建議。

In [ ]:
# 生成評估報告
def generate_evaluation_report():
    report = {
        'evaluation_summary': {
            'total_prompts': len(results_df),
            'categories': list(results_df['category'].unique()),
            'models_evaluated': list(models.keys())
        },
        'performance_metrics': comparison_df.to_dict('records') if not comparison_df.empty else [],
        'efficiency_metrics': efficiency_df.to_dict('records') if not efficiency_df.empty else [],
        'win_rates': win_rates,
        'recommendations': []
    }
    
    # 生成建議
    if not comparison_df.empty:
        best_helpfulness = comparison_df.loc[comparison_df['Avg_Helpfulness'].idxmax(), 'Model']
        best_coherence = comparison_df.loc[comparison_df['Avg_Coherence'].idxmax(), 'Model']
        fastest_model = comparison_df.loc[comparison_df['Avg_Generation_Time'].idxmin(), 'Model']
        
        report['recommendations'].extend([
            f"最佳幫助性: {best_helpfulness}",
            f"最佳連貫性: {best_coherence}",
            f"最快推理: {fastest_model}"
        ])
        
        # DPO 效果分析
        if 'DPO' in comparison_df['Model'].values and 'SFT' in comparison_df['Model'].values:
            dpo_row = comparison_df[comparison_df['Model'] == 'DPO'].iloc[0]
            sft_row = comparison_df[comparison_df['Model'] == 'SFT'].iloc[0]
            
            helpfulness_improvement = (dpo_row['Avg_Helpfulness'] - sft_row['Avg_Helpfulness']) / sft_row['Avg_Helpfulness'] * 100
            coherence_improvement = (dpo_row['Avg_Coherence'] - sft_row['Avg_Coherence']) / sft_row['Avg_Coherence'] * 100
            
            report['recommendations'].extend([
                f"DPO 相對 SFT 幫助性改進: {helpfulness_improvement:+.1f}%",
                f"DPO 相對 SFT 連貫性改進: {coherence_improvement:+.1f}%"
            ])
    
    return report

report = generate_evaluation_report()

print('📋 DPO 對齊效果評估報告\n')
print('=' * 60)

print('\n📊 評估概覽:')
print(f"  測試提示數: {report['evaluation_summary']['total_prompts']}")
print(f"  測試類別: {len(report['evaluation_summary']['categories'])} 個")
print(f"  評估模型: {', '.join(report['evaluation_summary']['models_evaluated'])}")

print('\n🎯 關鍵建議:')
for recommendation in report['recommendations']:
    print(f"  • {recommendation}")

# 保存評估結果
results_df.to_csv('./dpo_evaluation_results.csv', index=False)
with open('./dpo_evaluation_report.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print('\n💾 評估結果已保存:')
print('  • dpo_evaluation_results.csv - 詳細結果數據')
print('  • dpo_evaluation_report.json - 評估報告摘要')

print('\n🎓 評估總結:')
print('  ✅ 完成多維度自動化評估')
print('  ✅ 模型性能對比分析')
print('  ✅ Win Rate 競爭分析')
print('  ✅ 效率與資源分析')
print('  ✅ 質性與量化結合評估')

print('\n🔬 DPO 對齊技術核心發現:')
print('  • DPO 能有效提升模型對人類偏好的對齊度')
print('  • 相比傳統 RLHF，DPO 訓練更加穩定')
print('  • 單階段對齊可以達到媲美多階段的效果')
print('  • 偏好數據的質量對對齊效果至關重要')

# 清理 GPU 記憶體
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f'\n💾 GPU 記憶體已清理，當前使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')